# Breakdown analysis of life cycle GHG emissions for LIB raw materials

In [5]:
%run _imports.ipynb

In [6]:
BW_PROJECT = 'lib_rm' # insert your project name here
bw.projects.set_current(BW_PROJECT)

EI_DB = 'ecoinvent-3.10-cutoff' # name of ecoinvent database in your project
LIB_RM_DB = "EV battery metals"

IMPACT_METHODS = {"Climate change": ('IPCC 2021', 'climate change: including SLCFs', 'global warming potential (GWP100)')}

# (name, reference product, location)
INVENTORIES = {
    "Lithium, brine":        ("lithium hydroxide production, Salar de Atacama", "lithium hydroxide, battery grade", "CL"),
    "Lithium, spodumene":    ("lithium hydroxide production, from Australian spodumene", "lithium hydroxide, battery grade", "CN"),
    "Cobalt":                ("cobalt sulfate production, from copper-cobalt ore, economic allocation", "cobalt sulfate", "CN"),
    "Nickel":                ("nickel sulfate production, average excluding China, economic allocation", "nickel sulfate", "GLO"),
    "Graphite, natural":     ("natural graphite production, battery grade, from Heilongjiang", "natural graphite, battery grade", "CN")
}

### Total life cycle GHG emissions

In [7]:
total_ghg_emissions = {}

for rm in INVENTORIES:
    rm_ds = [ds for ds in bw.Database(LIB_RM_DB)
             if ds['name'] == INVENTORIES[rm][0] 
             and ds['reference product'] == INVENTORIES[rm][1]
             and ds['location'] == INVENTORIES[rm][2]][0]
    
    impacts = supporting_functions.multi_lcia(rm_ds, IMPACT_METHODS)
    total_ghg_emissions[rm] = impacts

In [8]:
total_ghg_emissions = pd.DataFrame(total_ghg_emissions).T
total_ghg_emissions

,Climate change
"Lithium, brine",7.702184
"Lithium, spodumene",17.237323
Cobalt,14.990897
Nickel,5.479065
"Graphite, natural",10.560489


In [9]:
# Export total GHG emissions
total_ghg_emissions.to_csv(DATA_DIR / "results" / f"fig2_total_ghgs_{datetime.datetime.today().strftime('%d-%m-%Y')}.csv", index_label="Metal")

### GHG emissions breakdown analysis

In [10]:
CONTRIBUTORS_LIST = [
    "Electricity consumption",
    "Process heating",
    "Fuels consumption",
    "Reagents consumption",
    "Process emissions",
    "Other"
    ]

In [11]:
breakdown_lists = supporting_functions.get_breakdown_lists()

In [12]:
inventories_breakdown = pd.read_excel(INVENTORIES_PATH, sheet_name="Datasets for breakdown", index_col=0, skiprows=1)
inventories_breakdown

,name,reference product,location,amount,stage
metal,,,,,
"Lithium, brine","lithium hydroxide production, Salar de Atacama","lithium hydroxide, battery grade",CL,1.000000,Refining
"Lithium, brine","lithium carbonate production, Salar de Atacama","lithium carbonate, battery grade",CL,1.050000,Refining
"Lithium, brine","lithium brine purification, Salar de Atacama",purified lithium brine,CL,22.884815,Concentration
"Lithium, brine","lithium brine production, from evaporation pon...","lithium brine, from evaporation pond",CL,3.862008,Mining
"Lithium, spodumene","lithium hydroxide production, from Australian ...","lithium hydroxide, battery grade",CN,1.000000,Refining
"Lithium, spodumene",spodumene concentrate production,spodumene concentrate,AU,6.420000,Concentration
Cobalt,"cobalt sulfate production, from copper-cobalt ...",cobalt sulfate,CN,1.000000,Refining
Cobalt,"cobalt hydroxide, hydrometallurigcal procesing...",cobalt hydroxide,CD,1.100000,Concentration
Cobalt,"copper-cobalt mining, industrial",copper-cobalt ore,CD,50.600000,Mining


In [13]:
impacts_breakdown = {}

for rm in list(set(inventories_breakdown.index)):
    print("##########################")
    print(rm)
    print("##########################")
    
    impacts_breakdown[rm] = {}
    skip_inventories = list(inventories_breakdown.loc[rm]["name"])

    for index, row in inventories_breakdown.loc[rm].iterrows():
        inv_amount = row["amount"]
        try:
            inv = [ds for ds in bw.Database(LIB_RM_DB)
                if ds['name'] == row["name"]
                and ds['reference product'] == row["reference product"]
                and ds['location'] == row["location"]][0]
        except IndexError:
            inv = [ds for ds in bw.Database(EI_DB)
                if ds['name'] == row["name"]
                and ds['reference product'] == row["reference product"]
                and ds['location'] == row["location"]][0]
         
        print(inv_amount, "|", inv)
        
        impacts_breakdown[rm][row["name"]] = supporting_functions.lcia_system_contribution(inv, skip_inventories, 
                IMPACT_METHODS, CONTRIBUTORS_LIST, breakdown_lists, activity_amount=inv_amount)

##########################
Cobalt
##########################
1.0 | 'cobalt sulfate production, from copper-cobalt ore, economic allocation' (kilogram, CN, None)
1.1 | 'cobalt hydroxide, hydrometallurigcal procesing of copper-cobalt ore, economic allocation' (kilogram, CD, None)
50.6 | 'copper-cobalt mining, industrial' (kilogram, CD, None)
##########################
Lithium, brine
##########################
1.0 | 'lithium hydroxide production, Salar de Atacama' (kilogram, CL, None)
1.05 | 'lithium carbonate production, Salar de Atacama' (kilogram, CL, None)
22.884814780185625 | 'lithium brine purification, Salar de Atacama' (kilogram, CL, None)
3.8620076761410544 | 'lithium brine production, from evaporation pond, Salar de Atacama' (kilogram, CL, None)
##########################
Lithium, spodumene
##########################
1.0 | 'lithium hydroxide production, from Australian spodumene' (kilogram, CN, None)
6.42 | 'spodumene concentrate production' (kilogram, AU, None)
################

In [14]:
ghgs_breakdown = {outer_key: {
    inner_key: sub_dict["Climate change"] 
    for inner_key, sub_dict in inner_dict.items()} 
    for outer_key, inner_dict in impacts_breakdown.items()
}

In [15]:
ghgs_breakdown_stages = {
        metal: { 
            stage: {
                contributor: 0 
                for contributor in CONTRIBUTORS_LIST} 
                for stage in ["Mining", "Concentration", "Refining"]}
            for metal in list(set(inventories_breakdown.index))
        }

for rm in ghgs_breakdown:
    total = 0
    for up in ghgs_breakdown[rm]:
        stage = inventories_breakdown.loc[rm][inventories_breakdown.loc[rm]["name"] == up]["stage"][0]
        for contributor in ghgs_breakdown[rm][up]:
            ghgs_breakdown_stages[rm][stage][contributor] += ghgs_breakdown[rm][up][contributor]

C:\Users\istrateir\AppData\Local\Temp\ipykernel_24132\849599517.py:13: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  stage = inventories_breakdown.loc[rm][inventories_breakdown.loc[rm]["name"] == up]["stage"][0]


In [16]:
ghgs_breakdown_stages_df = pd.DataFrame([(rm, stage, source, value)
                        for rm, stage in ghgs_breakdown_stages.items()
                        for stage, source in stage.items()
                        for source, value in source.items()],
                        columns=['Metal', 'Stage', 'Source', 'Value']).set_index("Metal")

In [17]:
ghgs_breakdown_stages_df

,Stage,Source,Value
Metal,,,
Cobalt,Mining,Electricity consumption,0.000000
Cobalt,Mining,Process heating,0.000000
Cobalt,Mining,Fuels consumption,2.958651
Cobalt,Mining,Reagents consumption,0.000000
Cobalt,Mining,Process emissions,0.000000
...,...,...,...
"Graphite, natural",Refining,Process heating,0.080017
"Graphite, natural",Refining,Fuels consumption,0.004292
"Graphite, natural",Refining,Reagents consumption,0.884381


In [18]:
# Export breakdown results
ghgs_breakdown_stages_df.to_csv(DATA_DIR / "results" / f"fig2_breakdown_ghgs_{datetime.datetime.today().strftime('%d-%m-%Y')}.csv", index_label="Metal")

In [19]:
# Sanity check, breakdown sum vs total emissions
for metal in ghgs_breakdown_stages:
    totals = 0
    for stage in ghgs_breakdown_stages[metal]:
        for contributor in ghgs_breakdown_stages[metal][stage]:
            totals += ghgs_breakdown_stages[metal][stage][contributor]
    print(metal, totals, "|", total_ghg_emissions.loc[metal].values[0], "| Diff:", totals - total_ghg_emissions.loc[metal].values[0])

Cobalt 14.990897342382507 | 14.990897206160097 | Diff: 1.3622240935262653e-07
Lithium, brine 7.702184735177559 | 7.702184411037539 | Diff: 3.241400197140365e-07
Lithium, spodumene 17.23732266082799 | 17.23732293458235 | Diff: -2.7375436317811364e-07
Nickel 5.479065122405995 | 5.479064855916995 | Diff: 2.664889997561204e-07
Graphite, natural 10.560489082067889 | 10.56048907223994 | Diff: 9.82794823300992e-09


### Breakdown GHG emissions from reagents consumption

In [30]:
reagents_impacts_breakdown = {}

for rm in list(set(inventories_breakdown.index)):
    print(rm)
    print("------------------------")
    reagents_impacts_breakdown[rm] = {}
    skip_inventories = list(inventories_breakdown.loc[rm]["name"])

    for index, row in inventories_breakdown.loc[rm].iterrows():
        inv_amount = row["amount"]
        try:
            inv = [ds for ds in bw.Database(LIB_RM_DB)
                if ds['name'] == row["name"]
                and ds['reference product'] == row["reference product"]
                and ds['location'] == row["location"]][0]
        except IndexError:
            inv = [ds for ds in bw.Database(EI_DB)
                if ds['name'] == row["name"]
                and ds['reference product'] == row["reference product"]
                and ds['location'] == row["location"]][0]

        reagents_impacts_breakdown[rm][row["name"]] = supporting_functions.lcia_reagents_disaggregation(
            inv, skip_inventories, IMPACT_METHODS, breakdown_lists, activity_amount=inv_amount)

Cobalt
------------------------
Lithium, brine
------------------------
Lithium, spodumene
------------------------
Nickel
------------------------
Graphite, natural
------------------------


In [31]:
reagents_ghgs_breakdown = {outer_key: {
    inner_key: sub_dict["Climate change"] 
    for inner_key, sub_dict in inner_dict.items()} 
    for outer_key, inner_dict in reagents_impacts_breakdown.items()
}

In [32]:
reagents_ghgs_breakdown_stages = {
        rm: { 
            stage: {
                contributor: 0 
                for contributor in breakdown_lists["reagent products"]} 
                for stage in ["Mining", "Concentration", "Refining"]}
            for rm in list(set(inventories_breakdown.index))
        }

for rm in reagents_ghgs_breakdown:
    total = 0
    for up in reagents_ghgs_breakdown[rm]:
        stage = inventories_breakdown.loc[rm][inventories_breakdown.loc[rm]["name"] == up]["stage"][0]
        for contributor in reagents_ghgs_breakdown[rm][up]:
            reagents_ghgs_breakdown_stages[rm][stage][contributor] += reagents_ghgs_breakdown[rm][up][contributor]

C:\Users\istrateir\AppData\Local\Temp\ipykernel_24132\3365614631.py:13: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  stage = inventories_breakdown.loc[rm][inventories_breakdown.loc[rm]["name"] == up]["stage"][0]


In [33]:
reagents_ghgs_breakdown_stages_df = pd.DataFrame([(metal, stage, reagent, value)
                        for metal, stage in reagents_ghgs_breakdown_stages.items()
                        for stage, reagent in stage.items()
                        for reagent, value in reagent.items()],
                        columns=['Metal', 'Stage', 'Reagent', 'Value']).set_index("Metal")

In [34]:
# Export reagents breakdown results
reagents_ghgs_breakdown_stages_df.to_csv(DATA_DIR / "results" / f"ESI_breakdown_reagents_ghgs_{datetime.datetime.today().strftime('%d-%m-%Y')}.csv", index_label="Metal")